In [1]:
from pathlib import Path
from collections import defaultdict
import numpy as np

data_dir = Path("data/")
dataset = "EN" # EN or ES

# Part 1(i): Emission scores
Compute $e(x|y) = \frac{\text{Count}(y \rightarrow x)}{\text{Count}(y)}$ and store it into a dictionary $f$.

In [2]:
def calculate_emission_scores(path):
    '''
    Inputs:
        path (Path object or str): path on the local directory to the dataset to load from.
    Outputs:
        f (dict): Key-value mapping of str(x, y) to log(e(x|y)) values.
    '''

    count_emission = defaultdict(int) # Stores Count(y -> x), where key is tuple (x, y), and value is Count(y -> x)
    count_labels = defaultdict(int) # Stores Count(y), where key is y
    
    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        
        # Process lines
        for line in lines:
            # Strip newline
            formatted_line = line.strip()
            
            # Only process lines that are not newlines
            if len(formatted_line) > 0:
                # Split into (x, y) pair
                split_data = formatted_line.split(" ")
                x, y = split_data[0].lower(), split_data[1]

                count_emission[(x, y)] += 1
                count_labels[y] += 1
    
    # Result dictionary that maps str(x, y) to log(e(x|y)) values
    f = {}
    
    # Estimate e(x|y) based on counts
    for x, y in count_emission:
        # Create str(x, y)
        feature_str = f"emission:{y}+{x}"
        e = count_emission[(x, y)] / count_labels[y]
        
        # Store score
        f[feature_str] = np.log(e)
    
    return f

# Part 1(ii): Transition scores
Compute $q(y_i|y_{i-1}) = \frac{\text{Count}(y_{i-1}, y_i)}{\text{Count}(y_{i-1})}$ and store it into a dictionary $f$.

In [3]:
def calculate_transition_scores(path):
    '''
    Inputs:
        path (Path object or str): path on the local directory to the dataset to load from.
    Outputs:
        f (dict): Key-value mapping of str(y_{i-1}, y_i) to log(q(y_i|y_{i-1})) values.
    '''
    
    count_transition = defaultdict(int) # Key is tuple (y_i, y_{i-1}), and value is Count(y_{i-1}, y_i)
    count_labels = defaultdict(int) # Stores Count(y_i), where key is y_i
    
    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        # Initialize prev_y as START
        prev_y = "START"
        
        # Process lines
        for line in lines:
            # Strip newline
            formatted_line = line.strip()
            
            # Only process lines that are not newlines
            if len(formatted_line) > 0:
                # Split into (x, y) pair
                split_data = formatted_line.split(" ")
                x, y = split_data[0].lower(), split_data[1]
                
                transition_key = (prev_y, y)
                count_transition[transition_key] += 1
                count_labels[y] += 1
                
                # Update for next word
                prev_y = y
            else:
                # End of sentence
                # Store Count(STOP|y_n)
                transition_key = (prev_y, "STOP")
                count_transition[transition_key] += 1
                
                # Start of next sentence: initialize prev_y to "START" and store Count(START)
                prev_y = "START"
                count_labels[prev_y] += 1
    
    # Result dictionary that maps str(x, y) to log(e(x|y)) values
    f = {}
    
    # Estimate e(x|y) based on counts
    for prev_y, y in count_transition:
        # Create str(y_{i-1}, y_i)
        feature_str = f"transition:{prev_y}+{y}"
        q = count_transition[(prev_y, y)] / count_labels[prev_y]
        
        # Store score
        f[feature_str] = np.log(q)
    
    return f

In [4]:
# Compute emission and transition parameters
f_emission_train = calculate_emission_scores(data_dir / dataset / "train")
f_transition_train = calculate_transition_scores(data_dir / dataset / "train")

# Combine the transition and emission dictionaries together
f = {**f_emission_train, **f_transition_train}
# Ensure the number of elements is correct
assert(len(f) == len(f_emission_train) + len(f_transition_train))

# Part 2(i)
Compute CRF scores for a given input and output sequence pair.

In [5]:
def compute_crf_score(x, y, feature_dict):
    ''' 
    Inputs:
        x (list[str]): Complete input word sentence (without START or STOP tags)
        y (list[str]): Complete output label sequence
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
    Outputs:
        p (float): Score given by p(y | x)
    '''
    
    # Input and output sequences must be of the same length
    assert(len(x) == len(y))
    n = len(x) # Sequence length
    
    # Stores the number of times each feature appears in (x, y)
    feature_count = defaultdict(int)
    
    # Compute emission features
    for i in range(n):
        formatted_word = x[i].lower()
        emission_key = f"emission:{y[i]}+{formatted_word}"
        feature_count[emission_key] += 1
    
    # Compute transition features
    # Add START and STOP tags to y
    updated_y = ["START"] + y + ["STOP"]
    for i in range(1, n+2):
        prev_y = updated_y[i-1]
        y_i = updated_y[i]
        transition_key = f"transition:{prev_y}+{y_i}"
        feature_count[transition_key] += 1
    
    # Compute score
    score = 0
    for feature_key, count in feature_count.items():
        weight = feature_dict[feature_key]
        score += weight * count
    
    return score

# Test function
x = "Great food with an awesome atmosphere !".split()
y = "O B-positive O O O B-positive O".split()
compute_crf_score(x, y, f)

-44.57667948595218

# Part 2(ii)
Viterbi algorithm for decoding.

In [6]:
def get_states(path):
    '''
    Inputs:
        path (Path object or str): path on the local directory to the dataset to load from.
    Outputs:
        states (list[str]): Unique states in the dataset.
    '''
    states = set()
    
    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        
        # Process lines
        for line in lines:
            # Strip newline
            formatted_line = line.strip()
            
            # Only process lines that are not newlines
            if len(formatted_line) > 0:
                # Split into (x, y) pair
                split_data = formatted_line.split(" ")
                x, y = split_data[0], split_data[1]
                states.add(y)
    
    return list(states)

def viterbi_decode(x, states, feature_dict, default_index=0):
    '''
    Inputs:
        x (list[str]): Input sequence.
        states (list[str]): Possible output states.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        default_index (int, optional): Index to default for backpointer if scores are nan.
    Outputs:
        y (list[str]): Most probable output sequence.
    '''
    
    n = len(x) # Number of words
    d = len(states) # Number of states
    scores = np.full((n, d), np.nan)
    bp = np.full((n, d), default_index, dtype=np.int) # TODO: Default to 'O', or something else?

    # Convert to lowercase
    x = [x[i].lower() for i in range(n)]
    
    # Compute START transition scores
    for i, current_y in enumerate(states):
        transition_key = f"transition:START+{current_y}"
        emission_key = f"emission:{current_y}+{x[0]}"
        if transition_key in feature_dict and emission_key in feature_dict:
            transmission_score = feature_dict[transition_key]
            emission_score = feature_dict[emission_key]
            scores[0, i] = transmission_score + emission_score
    
    # Recursively compute best scores based on transmission and emission scores at each node
    for i in range(1, n):
        for k, prev_y in enumerate(states):
            for j, current_y in enumerate(states):
                transition_key = f"transition:{prev_y}+{current_y}"
                emission_key = f"emission:{current_y}+{x[i]}"
                
                # Only consider if the feature exists
                if transition_key in feature_dict and emission_key in feature_dict:
                    transition_score = feature_dict[transition_key]
                    emission_score = feature_dict[emission_key]
                    overall_score = emission_score + transition_score + scores[i-1, k]
                    
                    # Better score is found: Update backpointer and score arrays
                    if (np.isnan(scores[i, j]) and not np.isnan(overall_score)) or overall_score > scores[i, j]:
                        scores[i, j] = overall_score
                        bp[i,j] = k
    
    # Compute for STOP
    highest_score = None
    highest_bp = default_index
    
    for j, prev_y in enumerate(states):
        if not np.isnan(scores[n-1, j]):
            transition_key = f"transition:{prev_y}+STOP"

            if transition_key in feature_dict:
                transition_score = feature_dict[transition_key]
                overall_score = transition_score + scores[n-1, j]
                if highest_score == None or overall_score > highest_score:
                    highest_score = overall_score
                    highest_bp = j
    
    # Follow backpointers to get output sequence
    result = [states[highest_bp]]
    prev_bp = highest_bp
    for i in range(n-1, 0, -1):
        prev_bp = bp[i, prev_bp]
        output = states[prev_bp]
        # Prepend result to output list
        result = [output] + result
    
    return result

states = get_states(data_dir / dataset / "train")
viterbi_decode(x, states, f) # Should be similar or equal to y

['O', 'B-positive', 'O', 'O', 'O', 'B-positive', 'O']

In [7]:
# Perform decoding on dev sets
def inference(path, states, feature_dict):
    '''
    Given a path, perform inference on sentences in the dataset and writes it to disk.
    
    Inputs:
        path (Path object or str): path on the local directory to the dataset to perform inference on.
        states (list[str]): Unique states that can be predicted.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
    Outputs:
        None
    '''
    
    default_index = states.index('O')
    sentences = []

    # Write predictions to file
    output_filename = str(path).replace(".in", ".p2.out")

    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        sentence = []
        
        for line in lines:
            formatted_line = line.strip()
            
            # Not the end of sentence, add it to the list
            if len(formatted_line) > 0:
                sentence.append(formatted_line)
            else:
                # End of sentence
                sentences.append(sentence)
                sentence = []
    
    # Write output file
    with open(output_filename, "w") as wf:
        for sentence in sentences:
            # Run predictions
            pred_sentence = viterbi_decode(sentence, states, feature_dict, default_index)
            
            # Write original word and predicted tags
            for i in range(len(sentence)):
                wf.write(sentence[i] + " " + pred_sentence[i] + "\n")
            
            # End of sentence, write newline
            wf.write("\n")

inference(data_dir / dataset / "dev.in", states, f)

# Part 3(i)
Loss calculation using forward algorithm. We first define the score calculation functions for a single sample.

In [8]:
def compute_forward_score(x, feature_dict, states):
    '''
    Uses the forward algorithm to compute the score for a given sequence.
    
    Inputs:
        x (list[str]): Input sequence.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        forward_score (float): Forward score for this sequence.
    '''
    
    n = len(x) # Number of words
    d = len(states) # Number of states
    scores = np.full((n, d), np.nan)

    # Convert to lowercase
    x = [x[i].lower() for i in range(n)]
    
    # Compute START transition scores
    for i, current_y in enumerate(states):
        transition_key = f"transition:START+{current_y}"
        emission_key = f"emission:{current_y}+{x[0]}"
        if transition_key in feature_dict and emission_key in feature_dict:
            transmission_score = feature_dict[transition_key]
            emission_score = feature_dict[emission_key]
            # Sum exponentials
            scores[0, i] = np.exp(transmission_score + emission_score)
    
    # Recursively compute best scores based on transmission and emission scores at each node
    for i in range(1, n):
        for k, prev_y in enumerate(states):
            for j, current_y in enumerate(states):
                transition_key = f"transition:{prev_y}+{current_y}"
                emission_key = f"emission:{current_y}+{x[i]}"
                
                # Only consider if the feature exists
                if transition_key in feature_dict and emission_key in feature_dict:
                    transition_score = feature_dict[transition_key]
                    emission_score = feature_dict[emission_key]
                    
                    # Sum exponentials
                    overall_score = np.exp(emission_score + transition_score) * scores[i-1, k]
                    
                    # Better score is found: Add to score array
                    if not np.isnan(overall_score):
                        if np.isnan(scores[i, j]):
                            scores[i, j] = 0
                        
                        scores[i, j] += overall_score
    
    # Compute for STOP
    forward_score = 0
    
    for j, prev_y in enumerate(states):
        if not np.isnan(scores[n-1, j]):
            transition_key = f"transition:{prev_y}+STOP"

            if transition_key in feature_dict:
                transition_score = feature_dict[transition_key]
                
                # Sum exponentials
                overall_score = np.exp(transition_score) * scores[n-1, j]
                forward_score += overall_score
    
    # Take log over entire sum
    forward_score = np.log(forward_score)
    
    return forward_score


def compute_crf_loss_sample(x, y, feature_dict, states):
    '''
    Inputs:
        x (list[str]): Input sequence.
        y (list[str]): Groundtruth output sequence.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        loss (float): Loss value for a single sample.
    '''
    first_term = compute_crf_score(x, y, feature_dict)
    forward_score = compute_forward_score(x, feature_dict, states)
    
    loss = -(first_term - forward_score)
    return loss

We use the methods defined early to compute the loss across the entire dataset.

In [9]:
def get_dataset(path):
    '''
    Given a path, load the data from file.
    
    Inputs:
        path (Path object or str): path on the local directory to the dataset to perform inference on.
    Outputs:
        input_sequences (list[list[str]]): List of input sequences
        input_labels (list[list[str]]): List of input labels
    '''
    input_sequences = []
    input_labels = []
    
    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        sentence = []
        labels = []
        
        for line in lines:
            formatted_line = line.strip()
            
            # Not the end of sentence, add it to the list
            if len(formatted_line) > 0:
                split_data = formatted_line.split(" ")
                x, y = split_data[0].lower(), split_data[1]

                sentence.append(x)
                labels.append(y)
            else:
                # End of sentence
                input_sequences.append(sentence)
                input_labels.append(labels)
                sentence = []
                labels = []
    
    return input_sequences, input_labels

def compute_crf_loss(input_sequences, input_labels, feature_dict, states):
    '''
    Given a path, perform inference on sentences in the dataset and writes it to disk.
    
    Inputs:
        input_sequences (list[list[str]]): List of input sequences
        input_labels (list[list[str]]): List of input labels
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        loss (float): Loss value for the dataset.
    '''
    loss = 0
    for i in range(len(input_sequences)):
        sample_loss = compute_crf_loss_sample(input_sequences[i], input_labels[i], feature_dict, states)
        loss += sample_loss
    
    return loss

f_zero = {k: -3 for k in f}
train_inputs, train_labels = get_dataset(data_dir / dataset / "train")
compute_crf_loss(train_inputs, train_labels, f, states)

2178.0848344696287

In [292]:
def forward_backward(x, feature_dict, states):
    '''
    Uses the backward algorithm to compute the score for a given sequence.
    
    Inputs:
        x (list[str]): Input sequence.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        backward_score (float): Backward score for this sequence.
    '''
    
    n = len(x) # Number of words
    d = len(states) # Number of states
    forward_scores = np.zeros((n, d))
    backward_scores = np.zeros((n, d))

    # Convert to lowercase
    x = [x[i].lower() for i in range(n)]

    # Start forward pass
    # Compute START transition scores
    for i, current_y in enumerate(states):
        transition_key = f"transition:START+{current_y}"
        emission_key = f"emission:{current_y}+{x[0]}"
        if transition_key in feature_dict and emission_key in feature_dict:
            transition_score = feature_dict[transition_key]
            emission_score = feature_dict[emission_key]
            # Sum exponentials
            forward_scores[0, i] = np.exp(transition_score + emission_score)
    
    # Recursively compute best scores based on transmission and emission scores at each node
    for i in range(1, n):
        for k, prev_y in enumerate(states):
            for j, current_y in enumerate(states):
                transition_key = f"transition:{prev_y}+{current_y}"
                emission_key = f"emission:{current_y}+{x[i]}"
                
                # Only consider if the feature exists
                if transition_key in feature_dict and emission_key in feature_dict:
                    transition_score = feature_dict[transition_key]
                    emission_score = feature_dict[emission_key]
                    # Sum exponentials
                    overall_score = np.exp(emission_score + transition_score) * forward_scores[i-1, k]

                    # Better score is found: Add to score array
                    forward_scores[i, j] += overall_score

    # Compute for STOP
    forward_prob = 0
    for j, prev_y in enumerate(states):
        transition_key = f"transition:{prev_y}+STOP"
        
        if transition_key in feature_dict:
            transition_score = feature_dict[transition_key]
            # Sum exponentials
            overall_score = np.exp(transition_score) * forward_scores[n-1, j]
            forward_prob += overall_score
    # End forward pass

    # Start backward pass
    # Compute STOP transition scores
    for i, current_y in enumerate(states):
        transition_key = f"transition:{current_y}+STOP"
        if transition_key in feature_dict: # and emission_key in feature_dict:
            transition_score = feature_dict[transition_key]
            # Sum exponentials
            backward_scores[n-1, i] = np.exp(transition_score)
    
    # Recursively compute best scores based on transmission and emission scores at each node
    for i in range(n-1, 0, -1):
        for k, next_y in enumerate(states):
            for j, current_y in enumerate(states):
                transition_key = f"transition:{current_y}+{next_y}"
                emission_key = f"emission:{next_y}+{x[i]}"
                
                # Only consider if the feature exists
                if transition_key in feature_dict and emission_key in feature_dict:
                    transition_score = feature_dict[transition_key]
                    emission_score = feature_dict[emission_key]
                    
                    # Sum exponentials
                    overall_score = np.exp(emission_score + transition_score) * backward_scores[i, k]
                    
                    # Better score is found: Add to score array
                    backward_scores[i-1, j] += overall_score
    
    # Compute for START
    backward_prob = 0
    
    for j, next_y in enumerate(states):
        transition_key = f"transition:START+{next_y}"
        emission_key = f"emission:{next_y}+{x[0]}" # Emission of last word

        if transition_key in feature_dict and emission_key in feature_dict:
            transition_score = feature_dict[transition_key]
            emission_score = feature_dict[emission_key]
            # Sum exponentials
            overall_score = np.exp(emission_score + transition_score) * backward_scores[0, j]
            backward_prob += overall_score
    # End backward pass
    
    # Ensure forward and backward prob are the same up to floating point errors
    assert(abs(forward_prob - backward_prob) < 1e-8)

    expected_counts = np.zeros((n, d))
    for i in range(n):
        for j, y in enumerate(states):
            expected_counts[i, j] = forward_scores[i, j] * backward_scores[i, j] / forward_prob
    
    feature_expected_counts = defaultdict(float)
    
    # Compute expected emission counts
    for i in range(n):
        for j, current_y in enumerate(states):
            emission_key = f"emission:{current_y}+{x[i]}"
            feature_expected_counts[emission_key] += expected_counts[i, j]
    
    # Compute expected START / STOP transition counts
    for j, next_y in enumerate(states):
        start_transition_key = f"transition:START+{next_y}"
        feature_expected_counts[start_transition_key] += expected_counts[0, j]
        
        stop_transition_key = f"transition:{next_y}+STOP"
        feature_expected_counts[stop_transition_key] += expected_counts[n-1, j]

    # Compute expected transition counts
    for i in range(0, n-1):
        for j, current_y in enumerate(states):
            for k, next_y in enumerate(states):
                transition_key = f"transition:{current_y}+{next_y}"
#                 feature_expected_counts[transition_key] += expected_counts[i, j] * expected_counts[i+1, k]              
                emission_key = f"emission:{current_y}+{x[i]}"
    
                if transition_key in feature_dict and emission_key in feature_dict:
                    transition_score = feature_dict[transition_key]
                    emission_score = feature_dict[emission_key]
                    overall_score = np.exp(transition_score + emission_score)
                    
                    denom = np.sum(forward_scores[i,:] * backward_scores[i,:])
                    transition_expected_counts = forward_scores[i, j] * backward_scores[i, k] * overall_score / denom
                    
                    assert(transition_expected_counts <= 1)

                    feature_expected_counts[transition_key] += transition_expected_counts
    
    return feature_expected_counts

forward_backward(['all', 'in', 'all', ',', 'the', 'food', 'was', 'great', '(', 'except', 'for', 'the', 'dessserts', ')', '.'], f, states)

defaultdict(float,
            {'emission:B-negative+(': 0.0,
             'emission:B-negative+)': 0.0,
             'emission:B-negative+,': 0.0,
             'emission:B-negative+.': 0.0,
             'emission:B-negative+all': 0.02260174720026268,
             'emission:B-negative+dessserts': 1.0000000000000002,
             'emission:B-negative+except': 0.0,
             'emission:B-negative+food': 0.16207416808999195,
             'emission:B-negative+for': 0.0,
             'emission:B-negative+great': 0.0,
             'emission:B-negative+in': 0.0,
             'emission:B-negative+the': 0.00027086185197281094,
             'emission:B-negative+was': 0.0,
             'emission:B-neutral+(': 0.0,
             'emission:B-neutral+)': 0.0,
             'emission:B-neutral+,': 0.0,
             'emission:B-neutral+.': 0.0,
             'emission:B-neutral+all': 0.0,
             'emission:B-neutral+dessserts': 0.0,
             'emission:B-neutral+except': 0.0,
             'emis

In [289]:
def get_feature_count(x, y, feature_dict):
    ''' 
    Inputs:
        x (list[str]): Complete input word sentence (without START or STOP tags)
        y (list[str]): Complete output label sequence
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
    Outputs:
        p (float): Score given by p(y | x)
    '''
    
    # Input and output sequences must be of the same length
    assert(len(x) == len(y))
    n = len(x) # Sequence length
    
    # Stores the number of times each feature appears in (x, y)
    feature_count = defaultdict(int)
    
    # Compute emission features
    for i in range(n):
        formatted_word = x[i].lower()
        emission_key = f"emission:{y[i]}+{formatted_word}"
        feature_count[emission_key] += 1
    
    # Compute transition features
    # Add START and STOP tags to y
    updated_y = ["START"] + y + ["STOP"]
    for i in range(1, n+2):
        prev_y = updated_y[i-1]
        y_i = updated_y[i]
        transition_key = f"transition:{prev_y}+{y_i}"
        feature_count[transition_key] += 1
    
    return feature_count

In [290]:
feature_gradients = defaultdict(float)
limit = 3

for i in range(limit):#len(train_labels)):
    x = train_inputs[i]
    y = train_labels[i]
    
    feature_expected_counts = forward_backward(x, f, states)
    actual_counts = get_feature_count(x, y, f)

    for k, v in feature_expected_counts.items():
        feature_gradients[k] += v

    for k, v in actual_counts.items():
        feature_gradients[k] -= v

In [291]:
# Check against numerical gradient is equal
# feature_key = 'emission:B-positive+food'
feature_key = 'transition:O+O'
new_f = f.copy()
delta = 1e-6

loss1 = compute_crf_loss(train_inputs[:limit], train_labels[:limit], f, states)
new_f[feature_key] += delta
loss2 = compute_crf_loss(train_inputs[:limit], train_labels[:limit], new_f, states)

numerical_gradient = (loss2 - loss1) / delta
analytical_gradient = feature_gradients[feature_key]
# Ensure numerical and analytical gradient are close
print(numerical_gradient, analytical_gradient)
assert(abs(numerical_gradient - analytical_gradient) <= 1e-3)

0.6420607405743795 -17.710934402902545


AssertionError: 